In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
backdoor = Backdoor(VggNet, config='16-layer', channels=3, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.CIFAR10, ImageEntity)

Files already downloaded and verified
Files already downloaded and verified


### Poison setup

In [3]:
patches = ImagePatch((9, 9), 3, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = (None, [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
loss = backdoor.train(2, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.99% | Loss: 1.8107 | Samples trained: 12496/50000
Epoch 1/3 | 49.98% | Loss: 1.6222 | Samples trained: 24992/50000
Epoch 1/3 | 74.98% | Loss: 1.4881 | Samples trained: 37488/50000
Epoch 1/3 | 99.97% | Loss: 1.3867 | Samples trained: 49984/50000
Epoch 1 complete | Loss: 1.3869
Epoch 2/3 | 24.99% | Loss: 0.9720 | Samples trained: 12496/50000
Epoch 2/3 | 49.98% | Loss: 0.9355 | Samples trained: 24992/50000
Epoch 2/3 | 74.98% | Loss: 0.8911 | Samples trained: 37488/50000
Epoch 2/3 | 99.97% | Loss: 0.8545 | Samples trained: 49984/50000
Epoch 2 complete | Loss: 0.8548
Epoch 3/3 | 24.99% | Loss: 0.6817 | Samples trained: 12496/50000
Epoch 3/3 | 49.98% | Loss: 0.6644 | Samples trained: 24992/50000
Epoch 3/3 | 74.98% | Loss: 0.6438 | Samples trained: 37488/50000
Epoch 3/3 | 99.97% | Loss: 0.6197 | Samples trained: 49984/50000
Epoch 3 complete | Loss: 0.6199
Training complete | Net Average Loss: 0.9539 | Total epochs: 3
Training started
Epoch 1/3 | 24.99% | Loss: 

(0.9538661183766524, 1.1215728756984076)

### Evaluating Backdoor

In [6]:
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 75.77% | Trojan 74.43% | Difference -1.34%
Base Accuracy on Poison 3.71% | Attack Success Rate (ASR): 65.55%
Average Tensor Distance: 300.15 | Net Tensor Difference 15007673.23


### Creating a Defense

In [7]:
defense = Defense(backdoor)

### Detection setup

In [8]:
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [9]:
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 75.16% | Original Accuracy 75.77% | Difference -0.61%
Patch size (5, 5) | Synthetic Poison Accuracy 74.88% | Original Accuracy 75.77% | Difference -0.89%
Patch size (7, 7) | Synthetic Poison Accuracy 73.47% | Original Accuracy 75.77% | Difference -2.30%
Patch size (9, 9) | Synthetic Poison Accuracy 72.85% | Original Accuracy 75.77% | Difference -2.92%
Patch size (11, 11) | Synthetic Poison Accuracy 70.77% | Original Accuracy 75.77% | Difference -5.00%
Average Accuracy 73.43% | Average Difference -2.34% | Lowest Score: 70.77% | Likihood of Backdoor: Low


### Detection Trojan

In [10]:
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 73.88% | Original Accuracy 74.43% | Difference -0.55%
Patch size (5, 5) | Synthetic Poison Accuracy 73.30% | Original Accuracy 74.43% | Difference -1.13%
Patch size (7, 7) | Synthetic Poison Accuracy 64.24% | Original Accuracy 74.43% | Difference -10.19%
Patch size (9, 9) | Synthetic Poison Accuracy 54.02% | Original Accuracy 74.43% | Difference -20.41%
Patch size (11, 11) | Synthetic Poison Accuracy 53.29% | Original Accuracy 74.43% | Difference -21.14%
Average Accuracy 63.75% | Average Difference -10.68% | Lowest Score: 53.29% | Likihood of Backdoor: High


### Blocking Patches setup

In [11]:
labels = {i : i - 1 if i > 0 else 9 for i in range(10)}
patch = ImagePatch((10, 10), 3, 'random')
n = 5000

### Blocking

In [12]:
blockedloader = defense.block(poisontest, patch, labels, n)

### Testing after Block Base

In [13]:
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
24.10% Testing complete | Loss: 1.2293 | Accuracy: 0.6000
48.19% Testing complete | Loss: 1.2662 | Accuracy: 0.5969
72.29% Testing complete | Loss: 1.2527 | Accuracy: 0.5995
96.39% Testing complete | Loss: 1.2339 | Accuracy: 0.6082
Testing complete | Loss: 1.2429 | Accuracy: 60.45%


### Testing after Block Trojan

In [14]:
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
24.10% Testing complete | Loss: 2.8102 | Accuracy: 0.3703
48.19% Testing complete | Loss: 2.9103 | Accuracy: 0.3445
72.29% Testing complete | Loss: 2.8936 | Accuracy: 0.3438
96.39% Testing complete | Loss: 2.9076 | Accuracy: 0.3434
Testing complete | Loss: 2.9017 | Accuracy: 34.42%


### Defense by retraining

In [15]:
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 0.7329 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 0.7219 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 0.7062 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 0.6718 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 0.6713
Epoch 2/3 | 24.95% | Loss: 0.7393 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 0.7222 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 0.7068 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 0.6712 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 0.6707
Epoch 3/3 | 24.95% | Loss: 0.7332 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 0.7194 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 0.7041 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 0.6695 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 0.6690
Training complete | Net Average Loss: 0.6703 | Total epochs: 3


### Evaluation after retraining

In [16]:
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 75.77% | Trojan 74.43% | Defense 74.58%
Accuracy on Posion | Base 3.71% | Defense 54.08% | Trojan ASR 65.55%
Difference from Baseline | Trojan -1.34% | Defense -1.19%
Defense Effectiveness | 11.47% decrease in ASR


### Pruning setup

In [30]:
defense.reset()
layers = ['layers.28', 'layers.26', 'layers.24', 'fc1']
amount = 0.7

### Pruning based on l-Infinity norm of weights from layers

In [31]:
defense.prune(layers, amount)

### Evaluation after prune

In [32]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 75.77% | Trojan 74.43% | Defense 16.03%
Accuracy on Posion | Base 3.71% | Defense 10.21% | Trojan ASR 65.55%
Difference from Baseline | Trojan -1.34% | Defense -59.74%
Defense Effectiveness | 55.34% decrease in ASR


### Retraining to recover

In [33]:
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 1.9777 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 1.9767 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 1.9752 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 1.9722 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.9722
Epoch 2/3 | 24.95% | Loss: 1.9785 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 1.9772 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 1.9759 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 1.9725 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 1.9725
Epoch 3/3 | 24.95% | Loss: 1.9804 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 1.9779 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 1.9763 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 1.9728 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 1.9728
Training complete | Net Average Loss: 1.9725 | Total epochs: 3


### Evaluation after retraining

In [34]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 75.77% | Trojan 74.43% | Defense 54.71%
Accuracy on Posion | Base 3.71% | Defense 28.71% | Trojan ASR 65.55%
Difference from Baseline | Trojan -1.34% | Defense -21.06%
Defense Effectiveness | 36.84% decrease in ASR
